# Construct Hamiltonian 

*Copyright (c) 2021 Institute for Quantum Computing, Baidu Inc. All Rights Reserved.*

## Outline

This tutorial will demonstrate how to construct a system's Hamiltonian using Quanlse. The outline of this tutorial is as follows:

- Introduction
- Preparation
- Define Hamiltonian
- Add control waveforms
- Simulation and related tools
- Summary

## Introduction

Generally, when modeling a superconducting qubits system, we need to define the system's Hamiltonian $\hat{H}_{\rm sys}$ firstly. The Hamiltonian, as the total energy, can be used to describe the behavior of entire system:
$$
\hat{H}_{\rm sys} = \hat{H}_{\rm drift} + \hat{H}_{\rm coup} + \hat{H}_{\rm ctrl}.
$$
It typically contains three terms - the time-independent drift term describing the individual qubits in the system, the coupling term describing the interaction between qubits, and the time-dependent control term describing the control driving acting on qubits.

After the Hamiltonian constructed, we can simulate the evolution of the quantum system by solving Schrödinger equation in the Heisenberg picture and then obtain the time-ordered evolution operator $U(t)$,

$$
i\hbar\frac{{\rm \partial}U(t)}{{\rm \partial}t} = \hat{H}(t)U(t).
$$

A variety of functions and a complete set of pre-defined operators are provided in `Quanlse`, which allows users to construct the Hamiltonian for large quantum systems with ease.

## Preparation 

After you installed Quanlse successfully, you can run the program below following this tutorial. Before running this particular program, you would need to import the necessary packages from Quanlse and a constant $\pi$ from the commonly-used Python library `math`:

In [ ]:
from Quanlse.QHamiltonian import QHamiltonian as QHam
from Quanlse.QOperator import number, driveX, driveY, duff
from Quanlse.QWaveform import gaussian, dragY1

from math import pi

## Define Hamiltonian

In this section, we will take a three-level superconducting quantum system consisting of two qubits to demonstrate how to construct a Hamiltonian using Quanlse. Now, we will create such a Hamiltonian in Quanlse by individually adding the time-independent drift terms, the coupling terms and finally the time-dependent control terms.

The first step is to instantiate a object 'ham' from class `QHam()`, which contains the basic information regarding the system. Its arguments include the number of qubits `subSysNum`, the system's energy level `sysLevel`, and the sampling-time `dt` (in nanoseconds).

In [ ]:
ham = QHam(subSysNum=2, sysLevel=3, dt=1.0)

Upon Quanlse 2.0.0, the class `QHam()` allows the users to define different energy levels for qubits in the same system. The users could pass the qubits' energy levels as a list to `sysLevel`. For instance, when `subSysNum=2`, we can pass `[2, 10]` to `sysLevel` to define a system that the qubit 0 has 2 energy levels and qubit 1 has 10. This system is defined in 20-dimensional space $\mathcal{H}^{2} \otimes \mathcal{H}^{10}$ ($\mathcal{H}^n$ indicates the $n$-dimensional Hilbert space).

Then, we add the drift Hamiltonian, which includes the detuning and the anharmonicity terms:
$$
\hat{H}_{\rm drift} = \sum_{i=0}^1(\omega_i-\omega_d)\hat{a}_i^\dagger\hat{a}_i + \sum_{i=0}^1 \frac{\alpha_i}{2} \hat{a}_i^{\dagger}\hat{a}_i^{\dagger}\hat{a}_i\hat{a}_i.
$$ 

These terms are added into `ham` by the function `addDrift()`. We pass in the according parameters and the operator which are pre-defined in the module `QOperator`. ([Clik here](https://quanlse.baidu.com/api/Quanlse/Quanlse.QOperator.html) to view all pre-defined operators)

In [ ]:
# The qubit frequency for the qubits, GHz * 2 * pi
wq = [4.887 * (2 * pi), 4.562 * (2 * pi)]  

# Anharmonicity for the qubits, GHz * 2 * pi
anharm = [- 0.317 * (2 * pi), - 0.317 * (2 * pi)]

# The drive pulse is in resonance  
wd = wq[0]

Since there are two qubits in this system, we can use a `for` loop of two `addDrift()` functions to add the drift terms. The first parameter is the operator which is a instance of class `QOperator`; `onSubSys` indicates which qubit(s) the term corresponds to; and `coef` is the coefficient for this system; `name` is a user-defined identifier of this term.

In [ ]:
# Add drift terms to the Hamiltonian
for q in range(2):
    ham.addDrift(number(3), onSubSys=q, coef=wq[q] - wd, name='number%d' % q)
    ham.addDrift(duff(3), onSubSys=q, coef=anharm[q] / 2, name='duff%d' % q)

Then, let us add the coupling terms describing the interaction between qubits into system.

## Add coupling
The coupling term can be written as:
$$
\hat{H}_{\rm coup} = \frac{g_{01}}{2} (\hat{a}_0\hat{a}_1^\dagger + \hat{a}_0^\dagger\hat{a}_1).
$$ 

In Quanlse, only one line of code is needed to add the coupling term - we use the function `addCoupling()`, which needs to select the qubits' indices `onSubSys` you want and specify a coupling strength `g`. Note that the argument `g` is the coefficient in the front of the coupling term, thus a constant $1/2$ is needed.

In [ ]:
ham.addCoupling(onSubSys=[0, 1], g=0.0277 * (2 * pi) / 2)

Lastly, we add the control terms.

## Add control waveforms
In a superconducting quantum system, the control Hamiltonian represents external control (microwave, magnetic flux etc.) acting on the qubits. To act X-Y control on qubit 0, the corresponding control terms are given as below:
$$
\hat{H}_{\rm ctrl} = A_0^x(t)\frac{\hat{a}_0+\hat{a}_0^\dagger}{2} + iA_0^y(t)\frac{\hat{a}_0-\hat{a}_0^\dagger}{2}. 
$$ 
In Quanlse, users can use function `addWave()` to define and add wave to system. Additionally, users can use `clearWaves()` to clear all waveforms in the specified control term.

Here, we will take `addWave()` to define and add the control wave. Each waveform function $A(t)$ can be defined using four arguments: start time `t0`, duration `t` and the corresponding parameters `a` 、`tau` 、`sigma`. The function `addWave()` allows us to define waveforms in different ways:

- **using preset waveform functions:**
Users can use the preset waveforms, here we use the gaussian wave `gaussian` as an example. The supported waveforms are listed here: [API](https://quanlse.baidu.com/api/Quanlse/Quanlse.QWaveform.html).
```python
ham.addWave(driveX(3), onSubSys=0, waves=gaussian(t0=0, t=20, a=1.1, tau=10, sigma=4), name = 'q0-ctrlx')
```

- **using user-defined wave functions:**
Users can also define a function in the form of `func(_t, args)`, where the first parameter `_t` is the time duration and `args` is the pulse parameters.
```python
def userWaveform(t0: Union[int, float], t: Union[int, float], a: float, tau: float, sigma: float,
             omega: float = None, phi: float = None) -> QWaveform:
    """
    Return a QWaveform object of user-defined wave.
    """

    def func(_t, args):
        _a, _tau, _sigma = args
        if _sigma == 0:
            return 0
        pulse = _a * exp(- ((_t - _tau) ** 2 / (2 * _sigma ** 2)))
        return pulse

    wave = QWaveform(f=func, t0=t0, t=t, args=(a, tau, sigma), omega=omega, phi=phi)
    wave.name = "user-defined wave"
    return wave
ham.addWave(driveX(3), onSubSys=0, waves=userWaveform(t0=0, t=20, a=1.1, tau=10, sigma=4), name = 'q0-ctrlx')
```

In this example, we add the pre-defined Gaussian and DRAG waveforms to the X and Y control terms respectively:

In [ ]:
ham.addWave(driveX(3), onSubSys=0, waves=gaussian(t0=0, t=20, a=1.1, tau=10, sigma=4))
ham.addWave(driveY(3), onSubSys=0, waves=dragY1(t0=0, t=20, a=1.7, tau=10, sigma=4))

## Simulation and related tools

With the Hamiltonian and control pulses defined, we can use the function `simulate()` to calculate the time-ordered evolution operator:

In [ ]:
results = ham.simulate()
print(results.result)

We can also display the detailed information regarding the constructed Hamiltonian using the function `print()` :

In [ ]:
print(ham)

The pulse sequence for each control term can be extracted by `getPulseSequences()` in `job`:

In [ ]:
print(ham.job.generatePulseSequence(driveY(3), 0))

To visualize the pulse sequences, use `plot()`:

In [ ]:
ham.job.plot()

## Summary

The purpose of this tutorial is to introduce how to construct the system Hamiltonian using Quanlse and to simulate and visualize it. After reading this tutorial, users can try different parameter values or functions given in this tutorial to gain a deeper understanding.